In [2]:
import pandas as pd
import numpy as np
from json import loads
from datetime import datetime
from unicodedata import normalize
from string import punctuation

In [3]:
def clean_txt(txt):
    return normalize('NFKD', txt)\
           .encode('ASCII', 'ignore')\
           .decode('utf-8')\
           .lower()\
           .strip()

def find_estado(x, cep):
    x = clean_txt(x)    
    if not x:
        return 'Não Informado'
    else:
        try:
            ix = np.where(cep.values == clean_txt(x))[0][0]
            return cep.iloc[ix].sigla.upper()
        except:
            return 'Estrangeiro'

def clean_location(x, cep, sigla):
    for s in sigla:
        if s in x.split(' '):
            return s.upper()
    
    for p in punctuation:
        x = x.replace(p, '|')
    
    x = clean_txt(x).split('|')
    if x == ['brasil'] or x == ['brazil']:
        return 'Brasileiro sem Estado'
    else:
        estado = 'Estrangeiro'
        for i in x:
            if find_estado(i, cep) != 'Estrangeiro':
                estado = find_estado(i, cep)
    
        return estado

In [4]:
a = loads(open('twitter.json').read())
a[0]

{'created_at': 'Sun Oct 07 23:59:59 +0000 2018',
 'id': 1049086971212156933,
 'id_str': '1049086971212156933',
 'full_text': 'Vamos subir à hashtag.\n#CiroNazista\n\n#17Neles',
 'truncated': False,
 'display_text_range': [0, 45],
 'entities': {'hashtags': [{'text': 'CiroNazista', 'indices': [23, 35]},
   {'text': '17Neles', 'indices': [37, 45]}],
  'symbols': [],
  'user_mentions': [],
  'urls': []},
 'metadata': {'iso_language_code': 'pt', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 982403477774184448,
  'id_str': '982403477774184448',
  'name': '🐸 @RobertaeDavid 17🔰',
  'screen_name': 'daviderobertas2',
  'location': 'Belo Horizonte, Brasil',
  'description': 'Cristãos, conservadores e defensores da família.',
  'url': None,


In [5]:
columns = ['created_at', 'full_text', 'entities', 'user', 'retweet_count', 'favorite_count']
candidates = ['fraudenasurnaseletronicas', 'ficatemer', 'viraviraciro','17neles', 'elenao']

In [6]:
len(a)

142438

In [7]:
df = pd.DataFrame(a)
df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,place,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,source,truncated,user
0,None,None,Sun Oct 07 23:59:59 +0000 2018,"[0, 45]","{'hashtags': [{'text': 'CiroNazista', 'indices...",NaN,0,False,Vamos subir à hashtag.\n#CiroNazista\n\n#17Neles,None,...,None,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 982403477774184448, 'id_str': '98240347..."
1,None,None,Sun Oct 07 23:59:59 +0000 2018,"[0, 42]","{'hashtags': [{'text': 'FicaTemer', 'indices':...","{'media': [{'id': 1049086968116776961, 'id_str...",0,False,"partiu austrália, beijos brasil\n#FicaTemer ht...",None,...,None,False,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 3432938446, 'id_str': '3432938446', 'na..."
2,None,None,Sun Oct 07 23:59:59 +0000 2018,"[0, 232]","{'hashtags': [{'text': '17Neles', 'indices': [...",NaN,1,False,Você pode aceitar continuar sendo roubado e em...,None,...,None,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 4832996920, 'id_str': '4832996920', 'na..."
3,None,None,Sun Oct 07 23:59:59 +0000 2018,"[0, 43]","{'hashtags': [{'text': 'Haddad13', 'indices': ...",NaN,1,False,Simbora #Haddad13 #HaddadPresidente #EleNão,None,...,None,NaN,NaN,NaN,NaN,1,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 991757309914345472, 'id_str': '99175730..."
4,None,None,Sun Oct 07 23:59:59 +0000 2018,"[16, 191]","{'hashtags': [{'text': 'FicaTemer', 'indices':...",NaN,8,False,"@o_colecionador Cara, o que vc acha do movimen...",None,...,None,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,"{'id': 320347999, 'id_str': '320347999', 'name..."


In [8]:
df = df[columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142438 entries, 0 to 142437
Data columns (total 6 columns):
created_at        142438 non-null object
full_text         142438 non-null object
entities          142438 non-null object
user              142438 non-null object
retweet_count     142438 non-null int64
favorite_count    142438 non-null int64
dtypes: int64(2), object(4)
memory usage: 6.5+ MB


In [9]:
cep = pd.read_csv('estados+cidades.csv', delimiter='|', header=None, names=['estado', 'sigla', 'cidade'])
cep = cep.applymap(lambda x: clean_txt(x))
cep.head()

,estado,sigla,cidade
0,acre,ac,acrelandia
1,acre,ac,assis brasil
2,acre,ac,brasileia
3,acre,ac,bujari
4,acre,ac,capixaba


In [10]:
sigla = set(cep.sigla.values.tolist())

In [11]:
df['location'] = df.user.apply(lambda x: x['location'])
df['estado'] = df.location.apply(lambda x: clean_location(x, cep, sigla))
df['user'] = df.user.apply(lambda x: x['name'])
df['entities'] = df.entities.apply(lambda x: ';'.join([i['text'] for i in x['hashtags']]))
df['candidate'] = df.entities.apply(lambda x: next((i for i in candidates if i in clean_txt(x)), 'branco'))
df['date'] = df.created_at.apply(lambda x : datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y').strftime('%Y-%m-%d %H:%M:%S'))
del df['created_at']
df.head()

,full_text,entities,user,retweet_count,favorite_count,location,estado,candidate,date
0,Vamos subir à hashtag.\n#CiroNazista\n\n#17Neles,CiroNazista;17Neles,🐸 @RobertaeDavid 17🔰,0,0,"Belo Horizonte, Brasil",MG,17neles,2018-10-07 23:59:59
1,"partiu austrália, beijos brasil\n#FicaTemer ht...",FicaTemer,lucas,0,0,,Não Informado,ficatemer,2018-10-07 23:59:59
2,Você pode aceitar continuar sendo roubado e em...,17Neles,Thais 🍃,0,1,"Arenápolis, Brasil",MT,17neles,2018-10-07 23:59:59
3,Simbora #Haddad13 #HaddadPresidente #EleNão,Haddad13;HaddadPresidente;EleNão,#FicaLeãoNaPrimeiraDivisão 🦁 #EleNão,1,1,"Pernambuco, Brasil",PE,elenao,2018-10-07 23:59:59
4,"@o_colecionador Cara, o que vc acha do movimen...",FicaTemer,Tião do Feijão,0,8,"Cesário Lange, Brasil",SP,ficatemer,2018-10-07 23:59:59


In [12]:
df.tail()

,full_text,entities,user,retweet_count,favorite_count,location,estado,candidate,date
142433,#EleNão #ELENUNCA https://t.co/lwXzuoAdCr,EleNão;ELENUNCA,leticiα #ELENÃO 1⃣3⃣,0,1,f.r.i.e.n.d.s,Estrangeiro,elenao,2018-10-07 00:00:04
142434,@katyperry #EleNão,EleNão,sabrina,0,0,Brasil,Brasileiro sem Estado,elenao,2018-10-07 00:00:04
142435,Padre Paulo Roberto de Oliveira adere ao #EleN...,EleNão,† icatolica. com,0,0,"São Luís, Maranhão",MA,elenao,2018-10-07 00:00:04
142436,"Amanhã não esqueça. Ou vc vota Ciro 12, ou que...",Elenao;Elenunca;eleições2018;ciro12,Ivan Brazil,0,0,"Sorocaba, Brasil",SP,elenao,2018-10-07 00:00:02
142437,#ViraViraCIRO #ciro12 #CiroSim https://t.co/KO...,ViraViraCIRO;ciro12;CiroSim,Andrea Pereira,1,0,,Não Informado,viraviraciro,2018-10-07 00:00:02


In [13]:
df.groupby("candidate")['user'].count()

candidate
17neles                      39353
elenao                       52839
ficatemer                    18164
fraudenasurnaseletronicas    18039
viraviraciro                 14043
Name: user, dtype: int64

In [14]:
df.groupby("estado")['user'].count()

estado
AC                         299
AL                         594
AM                        1214
AP                         145
BA                        2449
Brasileiro sem Estado    10083
CE                        2875
DF                        2214
ES                        1726
Estrangeiro              22045
GO                        1792
MA                        1112
MG                        6274
MS                         621
MT                         474
Não Informado            43387
PA                        1320
PB                        1019
PE                        2907
PI                         472
PR                        3400
RJ                       13498
RN                        1062
RO                         254
RR                          81
RS                        4827
SC                        1595
SE                         488
SP                       13908
TO                         303
Name: user, dtype: int64

In [15]:
df['location'][df.estado=='Estrangeiro'].apply(lambda x: clean_txt(x)).value_counts()[:20]

021                 324
na sua              217
hogwarts            183
na merda            172
narnia              161
bh                  138
fan account         127
portugal            112
027                 109
no seu coracao      107
london, england      98
inferno              95
fundo do poco        83
lisboa, portugal     82
argentina            82
los angeles, ca      77
paris, france        77
terra do nunca       70
hell                 69
rio                  67
Name: location, dtype: int64

In [16]:
df.describe()

,retweet_count,favorite_count
count,142438.000000,142438.000000
mean,3.433473,9.182648
std,87.799505,284.844025
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,1.000000,2.000000
max,19063.000000,72731.000000


In [17]:
columns = ['date', 'user', 'candidate', 'estado', 'location', 'retweet_count', 'favorite_count', 'full_text']

In [18]:
df.date = df.date.astype('datetime64')
df.candidate = df.candidate.astype('category').cat.rename_categories(['#17Neles', '#EleNão', '#FicaTemer', '#FraudeNasUrnasEletrônicas',
       '#ViraViraCIRO'])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142438 entries, 0 to 142437
Data columns (total 9 columns):
full_text         142438 non-null object
entities          142438 non-null object
user              142438 non-null object
retweet_count     142438 non-null int64
favorite_count    142438 non-null int64
location          142438 non-null object
estado            142438 non-null object
candidate         142438 non-null category
date              142438 non-null datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(2), object(5)
memory usage: 8.8+ MB


In [20]:
df = df[columns].sort_values(['date']).reset_index(drop=True)
df.head()

,date,user,candidate,estado,location,retweet_count,favorite_count,full_text
0,2018-10-07 00:00:02,Andrea Pereira,#ViraViraCIRO,Não Informado,,1,0,#ViraViraCIRO #ciro12 #CiroSim https://t.co/KO...
1,2018-10-07 00:00:02,Ivan Brazil,#EleNão,SP,"Sorocaba, Brasil",0,0,"Amanhã não esqueça. Ou vc vota Ciro 12, ou que..."
2,2018-10-07 00:00:04,sabrina,#EleNão,Brasileiro sem Estado,Brasil,0,0,@katyperry #EleNão
3,2018-10-07 00:00:04,† icatolica. com,#EleNão,MA,"São Luís, Maranhão",0,0,Padre Paulo Roberto de Oliveira adere ao #EleN...
4,2018-10-07 00:00:04,leticiα #ELENÃO 1⃣3⃣,#EleNão,Estrangeiro,f.r.i.e.n.d.s,0,1,#EleNão #ELENUNCA https://t.co/lwXzuoAdCr


In [21]:
df.to_json('dataset.json.gzip', compression='gzip')

In [22]:
df.to_csv('dataset2.csv', sep=';', index=False, encoding='utf-8')

In [52]:
df.replace(';',',').drop(['full_text', 'user'], axis=1).to_csv('dataset2.csv', sep=';', index_label="twitter", encoding='utf-8')